In [0]:
spark.conf.set("fs.azure.account.key.storage1998de.dfs.core.windows.net", "C64g9Q7x9IFzTjhaK/ffnmou2xW7nDCjOL17jI/EyXaL/r1N0WVOKiLyMKCLYT3WlHmpLpp82plY+AStmRCOKg==")

In [0]:
dbutils.fs.ls("abfss://raw@storage1998de.dfs.core.windows.net/")

[FileInfo(path='abfss://raw@storage1998de.dfs.core.windows.net/muneebuddin-adil/', name='muneebuddin-adil/', size=0, modificationTime=1727234318000)]

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.appName("ReadData").getOrCreate()
storage_account_name = "storage1998de"
src_container = "raw"
file_path = "muneebuddin-adil/Healthcare/refs/heads/main/healthcare_dataset.csv"
target_container = "cleaned"

adls_url = f"abfss://{src_container}@{storage_account_name}.dfs.core.windows.net/{file_path}"

df = spark.read.csv(adls_url, sep=',', header=True,inferSchema=True)

df.show()

df.createOrReplaceTempView("healthdataset")

+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
|       LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|            

In [0]:
df = df.withColumnRenamed("Medical Cc", "Medical_Condition")\
       .withColumnRenamed("Billing Amount", "Billing_Amount")\
       .withColumnRenamed("Room Number", "Room_Number")\
       .withColumnRenamed("Admission Type", "Admission_Type")\
       .withColumnRenamed("Discharge Date", "Discharge_Date")\
       .withColumnRenamed("Test Results", "Test_Results")

In [0]:

# Drop duplicates based on all columns
df_no_duplicates = df.dropDuplicates()

# Count the number of records in the original and the new DataFrame
if df.count() != df_no_duplicates.count():
    print("There are duplicates in the DataFrame.")
else:
    print("No duplicates found.")

There are duplicates in the DataFrame.


In [0]:
for a in df.columns:
    if df.filter(df[a].isNull()):
        null_count = df.filter(df[a].isNull()).count()    
print('The number of Null values are : ',null_count)

The number of Null values are :  0


In [0]:
df.write.mode('overwrite').csv(f"abfss://{target_container}@storage1998de.dfs.core.windows.net/cleaned",header=True)

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime

spark = SparkSession.builder.appName("ReadData").getOrCreate()
storage_account_name = "storage1998de"
src_container = "cleaned"
file_path = "/cleaned/part-*"
target_container = "transformed"

adls_url = f"abfss://{src_container}@{storage_account_name}.dfs.core.windows.net/{file_path}"

temp_df = spark.read.csv(adls_url, sep=',', header=True,inferSchema=True)

temp_df.show()
temp_df.count()


+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|               Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|          Doctor|            Hospital|Insurance Provider|    Billing_Amount|Room_Number|Admission_Type|Discharge_Date| Medication|Test_Results|
+-------------------+---+------+----------+-----------------+-----------------+----------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+
|      Bobby JacksOn| 30|  Male|        B-|           Cancer|       2024-01-31|   Matthew Smith|     Sons and Miller|        Blue Cross|18856.281305978155|        328|        Urgent|    2024-02-02|Paracetamol|      Normal|
|       LesLie TErRy| 62|  Male|        A+|          Obesity|       2019-08-20| Samantha Davies|            

55500

In [0]:
transform_df = """SELECT * , row_number() over (order by Name) as ID FROM healthdataset"""
temp_df = spark.sql(transform_df)
temp_df.show(80)

+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---+
|              Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|              Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results| ID|
+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---+
|      AARON DuncAn| 22|  Male|       AB-|          Obesity|       2019-07-24|          Ryan Perry|          Welch-Yang|  UnitedHealthcare| 39906.14730805009|        279|        Urgent|    2019-08-04|Paracetamol|    Abnormal|  1|
|       AARON HicKS| 76|Female|        A+|        Arthritis|       2022-03-02|  

In [0]:
from pyspark.sql.functions import initcap
temp_df = temp_df.withColumn("Name", initcap(df["Name"]))
temp_df.show(80)

+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---+
|              Name|Age|Gender|Blood Type|Medical Condition|Date of Admission|              Doctor|            Hospital|Insurance Provider|    Billing Amount|Room Number|Admission Type|Discharge Date| Medication|Test Results| ID|
+------------------+---+------+----------+-----------------+-----------------+--------------------+--------------------+------------------+------------------+-----------+--------------+--------------+-----------+------------+---+
|      Aaron Duncan| 22|  Male|       AB-|          Obesity|       2019-07-24|          Ryan Perry|          Welch-Yang|  UnitedHealthcare| 39906.14730805009|        279|        Urgent|    2019-08-04|Paracetamol|    Abnormal|  1|
|       Aaron Hicks| 76|Female|        A+|        Arthritis|       2022-03-02|  

In [0]:
temp_df.write.mode('overwrite').option('header', True).parquet(f"abfss://{target_container}@storage1998de.dfs.core.windows.net/transformed")